In [ ]:
# BUAN 5310 - statistical learning
# Group project
# Group 11: Han Li - Wei Li - Shuai Ma - Huy Le
# Decision tree model

# Readme: This file include all the code and result for building decision tree model. The dataset was used is Cleaned_Data.csv
# which is full data after cleanning. In each Airline and Airport we will conduct full and pruned tree. 
# Then use the trained model for trainning accuracy, testing accuracy and k-fold validation
#

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import datetime

In [2]:
# getting data from full dataset
dat = pd.read_csv("../data/Cleaned_Data.csv")


In [3]:
#drop unused columns
dat = dat.drop(columns =["ID","FrequentDestination1",
                             "FrequentDestination2",
                             "FrequentDestination3",
                             "FrequentDestination4",
                             "FrequentDestination5",
                             "FrequentDestination6",
                             "FrequentDestination7",
                        "TotalDepartureHr","MileageAirline"], axis =1)

In [4]:
dat.describe()

,Airport,Airline,Age,Gender,Nationality,TripPurpose,TripDuration,FlyingCompanion,ProvinceResidence,GroupTravel,...,ModeTransport,AccessCost,AccessTime,Occupation,MileageAirline1,MileageAirline2,MileageAirline3,MileageAirline4,MileageAirline5,Income
count,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,...,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.000000,463.00000,463.000000,463.000000
mean,1.505400,2.425486,39.773218,1.524838,1.479482,1.501080,25.989201,2.781857,3.393089,1.835853,...,3.842333,11067.991361,62.498920,6.730022,0.161987,0.066955,0.030238,0.25270,0.488121,4.488121
std,0.500512,1.212471,13.462726,0.499923,1.012428,0.842267,71.328230,3.856452,2.615496,0.370809,...,2.406471,19481.481081,57.715149,4.174560,0.368837,0.250214,0.171426,0.43503,0.500400,2.535328
min,1.000000,1.000000,17.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000
25%,1.000000,1.000000,29.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,2.000000,...,2.000000,4000.000000,30.000000,2.000000,0.000000,0.000000,0.000000,0.00000,0.000000,2.000000
50%,2.000000,2.000000,37.000000,2.000000,1.000000,1.000000,5.000000,2.000000,3.000000,2.000000,...,3.000000,7000.000000,50.000000,8.000000,0.000000,0.000000,0.000000,0.00000,0.000000,4.000000
75%,2.000000,4.000000,50.000000,2.000000,1.000000,2.000000,8.000000,3.000000,5.000000,2.000000,...,6.000000,12642.000000,64.000000,12.000000,0.000000,0.000000,0.000000,1.00000,1.000000,8.000000
max,2.000000,4.000000,80.000000,2.000000,5.000000,4.000000,730.000000,34.000000,8.000000,2.000000,...,11.000000,350000.000000,390.000000,12.000000,1.000000,1.000000,1.000000,1.00000,1.000000,8.000000


In [5]:
# create data for airport model (Xap)
Xap = dat.loc[:, ~dat.columns.isin(['Airport'])]
yap = dat['Airport'].astype('category').cat.codes
yap.name= 'Airport' # this one is for name the tree graph

# prepare data for airline model (Xal)
Xal = dat.drop(columns =["Airline"], axis =1)
yal = dat['Airline'].astype('category').cat.codes
yal.name = "Airline" # this one is for name the tree graph

In [6]:
# decision tree function, return trained model
def decision_tree(X_train, y_train, maxdepth = None, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2 ):
    now =str(datetime.datetime.now())
    clf =tree.DecisionTreeClassifier(class_weight=None, 
                                     criterion ='gini', 
                                     max_depth =maxdepth, 
                                     max_features = max_feature, 
                                     max_leaf_nodes= maxleaf,
                                    min_samples_leaf = minsamleaf, min_samples_split =minsamsplit,
                                    min_weight_fraction_leaf =0.0, presort =False, random_state =100, splitter = 'best')
    clf = clf.fit(X_train, y_train)
    # tree graph will be generate with name: Airport/Airline datetime .dot
    tree.export_graphviz(clf,out_file=str(y_train.name)+now +".dot",feature_names=X_train.columns) 
    return(clf)

# testing decision tree model, return the accuracy
# multi = false => Y only have 0 or 1
def test_decisiont_tree(X_test, y_test, model, multi = False):
    y_pred =model.predict(X_test)
    if multi==True:        
        print(metrics.confusion_matrix(y_test,y_pred))
        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
        print(classification_report(y_test, y_pred, target_names=['Korean Air(KE)','Asiana Airlines','Korean LCC','Foreign Airlines']))
    else:
        print(metrics.confusion_matrix(y_test,y_pred))
        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
        print("Recall:", metrics.recall_score(y_test, y_pred))
        print("Precision:",metrics.precision_score(y_test, y_pred))
        print("F measure:",metrics.f1_score(y_test, y_pred))

# k-fold validation, X and y shoudl be full dataset
def kfold_test(model, X, y, k):
    kf = KFold(n_splits=k)
    kf.get_n_splits(X)
    acc= 0
    for train_index, test_index in kf.split(X):
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]
        X_train, y_train= X.iloc[train_index], y.iloc[train_index]
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc =acc + metrics.accuracy_score(y_test, y_pred)
        print(metrics.confusion_matrix(y_test,y_pred))
        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Average accuracy: ", acc/k) # return the average accuracy

## Model training
### Airport choice model

#### A. Default prameters

In [7]:
# train airport model with default parameter
X_train, X_test, y_train, y_test = train_test_split(Xap, yap, test_size=0.30,random_state=6)
airport_model_default = decision_tree(X_train, y_train)#, maxdepth = 10, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2 )

#testing model training accuracy
print('Trainning accuracy')
test_decisiont_tree(X_train,y_train,airport_model_default, False)#, maxdepth = 5,max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2)
print('Testing accuracy')
#testing with 30% testing
test_decisiont_tree(X_test,y_test,airport_model_default, False)

Trainning accuracy
[[160   0]
 [  0 164]]
Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F measure: 1.0
Testing accuracy
[[55 14]
 [13 57]]
Accuracy: 0.8057553956834532
Recall: 0.8142857142857143
Precision: 0.8028169014084507
F measure: 0.8085106382978723


#### B. With different parameter

In [21]:

X_train, X_test, y_train, y_test = train_test_split(Xap, yap, test_size=0.30,random_state=6)
airport_model_maxdepth8 = decision_tree(X_train, y_train, maxdepth = 8, max_feature=None, maxleaf =None, minsamleaf =1, minsamsplit = 2 )
#testing the result
print('Trainning accuracy')
print("maxdepth = 8, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2 ")
test_decisiont_tree(X_train,y_train, airport_model_maxdepth8, False)#, maxdepth = 5,max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2)

#testing with 30% testing
print('Testing accuracy')
test_decisiont_tree(X_test,y_test,airport_model_maxdepth8, False)

Trainning accuracy
maxdepth = 8, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2 
[[157   3]
 [  2 162]]
Accuracy: 0.9845679012345679
Recall: 0.9878048780487805
Precision: 0.9818181818181818
F measure: 0.9848024316109422
Testing accuracy
[[54 15]
 [11 59]]
Accuracy: 0.8129496402877698
Recall: 0.8428571428571429
Precision: 0.7972972972972973
F measure: 0.8194444444444444


### Airline choice model
#### A. with default parameters

In [22]:
# trainning airline model with default parameter
X_train, X_test, y_train, y_test = train_test_split(Xal,yal, test_size=0.30,random_state=6)
airline_model_default = decision_tree(X_train, y_train)#, maxdepth = 10, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2 )
# testing airline model with train set
print('Training accuracy')
test_decisiont_tree(X_train,y_train,airline_model_default, True)#, maxdepth = 5,max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2)

# testing airline model with test set
print('Testing accuracy')
test_decisiont_tree(X_test,y_test,airline_model_default, True)

Training accuracy
[[113   0   0   0]
 [  0  68   0   0]
 [  0   0  51   0]
 [  0   0   0  92]]
Accuracy: 1.0
                  precision    recall  f1-score   support

  Korean Air(KE)       1.00      1.00      1.00       113
 Asiana Airlines       1.00      1.00      1.00        68
      Korean LCC       1.00      1.00      1.00        51
Foreign Airlines       1.00      1.00      1.00        92

     avg / total       1.00      1.00      1.00       324

Testing accuracy
[[19  2  2 13]
 [ 9 10  2 13]
 [ 5  5 13  4]
 [10  8  4 20]]
Accuracy: 0.4460431654676259
                  precision    recall  f1-score   support

  Korean Air(KE)       0.44      0.53      0.48        36
 Asiana Airlines       0.40      0.29      0.34        34
      Korean LCC       0.62      0.48      0.54        27
Foreign Airlines       0.40      0.48      0.43        42

     avg / total       0.45      0.45      0.44       139



#### B. With different parameters

In [24]:
## trainning airline model with default parameter
X_train, X_test, y_train, y_test = train_test_split(Xal,yal, test_size=0.30,random_state=6)
airline_model_11_1_2 = decision_tree(X_train, y_train, maxdepth =11, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit =2 )

# testing airline model
print('Training accuracy')
print("maxdepth = 11, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2 ")
test_decisiont_tree(X_train,y_train,airline_model_11_1_2, True)#, maxdepth = 5,max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2)
# testing airline model with test set
print('Testing accuracy')
test_decisiont_tree(X_test,y_test,airline_model_11_1_2, True)

Training accuracy
maxdepth = 11, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2 
[[113   0   0   0]
 [  0  68   0   0]
 [  0   0  51   0]
 [  0   0   0  92]]
Accuracy: 1.0
                  precision    recall  f1-score   support

  Korean Air(KE)       1.00      1.00      1.00       113
 Asiana Airlines       1.00      1.00      1.00        68
      Korean LCC       1.00      1.00      1.00        51
Foreign Airlines       1.00      1.00      1.00        92

     avg / total       1.00      1.00      1.00       324

Testing accuracy
[[19  2  2 13]
 [ 9 10  2 13]
 [ 5  5 13  4]
 [10  8  4 20]]
Accuracy: 0.4460431654676259
                  precision    recall  f1-score   support

  Korean Air(KE)       0.44      0.53      0.48        36
 Asiana Airlines       0.40      0.29      0.34        34
      Korean LCC       0.62      0.48      0.54        27
Foreign Airlines       0.40      0.48      0.43        42

     avg / total       0.45      0.45      0.44       139



## Model testing (validation)
Method: K-fold

### Airport

In [14]:
#K =5
print('k=5')
kfold_test(airport_model_default, Xap, yap, 5)
#K =4
print('k=4')
kfold_test(airport_model_default, Xap, yap, 4)

k=5
[[64 29]
 [ 0  0]]
Accuracy: 0.6881720430107527
[[77 16]
 [ 0  0]]
Accuracy: 0.8279569892473119
[[28 15]
 [12 38]]
Accuracy: 0.7096774193548387
[[ 0  0]
 [24 68]]
Accuracy: 0.7391304347826086
[[ 0  0]
 [21 71]]
Accuracy: 0.7717391304347826
Average accuracy:  0.7473352033660589
k=4
[[91 25]
 [ 0  0]]
Accuracy: 0.7844827586206896
[[91 22]
 [ 0  3]]
Accuracy: 0.8103448275862069
[[ 0  0]
 [43 73]]
Accuracy: 0.6293103448275862
[[ 0  0]
 [20 95]]
Accuracy: 0.8260869565217391
Average accuracy:  0.7625562218890555


In [25]:
#K =5
print('k=5')
kfold_test(airport_model_maxdepth8, Xap, yap, 5)
#K =4
print('k=4')
kfold_test(airport_model_maxdepth, Xap, yap, 4)

k=5
[[65 28]
 [ 0  0]]
Accuracy: 0.6989247311827957
[[78 15]
 [ 0  0]]
Accuracy: 0.8387096774193549
[[29 14]
 [13 37]]
Accuracy: 0.7096774193548387
[[ 0  0]
 [21 71]]
Accuracy: 0.7717391304347826
[[ 0  0]
 [21 71]]
Accuracy: 0.7717391304347826
Average accuracy:  0.7581580177653109
k=4
[[88 28]
 [ 0  0]]
Accuracy: 0.7586206896551724
[[88 25]
 [ 0  3]]
Accuracy: 0.7844827586206896
[[ 0  0]
 [39 77]]
Accuracy: 0.6637931034482759
[[ 0  0]
 [18 97]]
Accuracy: 0.8434782608695652
Average accuracy:  0.7625937031484258


In [26]:
#K-fold testing Airline model, default parameter
# k =5
print('k=5')
kfold_test(airline_model_default, Xal, yal, 5)
#k=4
print('k=4')
kfold_test(airline_model_default, Xal, yal, 4)

k=5
[[18  6  7  2]
 [ 4  9  1  2]
 [ 7  2 15  5]
 [ 6  3  2  4]]
Accuracy: 0.4946236559139785
[[20  4  1  5]
 [ 4  9  2  9]
 [ 8  0 11  4]
 [ 2  0  7  7]]
Accuracy: 0.5053763440860215
[[10  5  4  6]
 [10 15  0 13]
 [ 0  1  3  1]
 [ 8  4  4  9]]
Accuracy: 0.3978494623655914
[[12  4  2 12]
 [ 5  3  0  5]
 [ 4  0  3  2]
 [14  8  2 16]]
Accuracy: 0.3695652173913043
[[17  6  0  8]
 [ 4  5  0  2]
 [ 3  0  8  1]
 [ 9  4  1 24]]
Accuracy: 0.5869565217391305
Average accuracy:  0.4708742402992052
k=4
[[23  7  4  6]
 [ 3 12  1  4]
 [ 4  3 25  4]
 [ 7  4  3  6]]
Accuracy: 0.5689655172413793
[[20  6  6  4]
 [ 5 13  8 10]
 [ 1  0 18  1]
 [ 7  3  7  7]]
Accuracy: 0.5
[[10  7  4 12]
 [ 8  8  1 14]
 [ 1  1  3  3]
 [16  9  2 17]]
Accuracy: 0.3275862068965517
[[15 10  0 15]
 [ 2  9  0  4]
 [ 6  0  5  3]
 [ 5  9  3 29]]
Accuracy: 0.5043478260869565
Average accuracy:  0.4752248875562219


In [18]:
#K-fold testing Airline model
# k =5
print('k=5')
kfold_test(airline_model_11_1_2, Xal, yal, 5)
#k=4
print('k=4')
kfold_test(airline_model_11_1_2, Xal, yal, 4)

k=5
[[17  3  4  9]
 [ 4  7  1  4]
 [ 7  2 15  5]
 [ 8  1  2  4]]
Accuracy: 0.46236559139784944
[[20  4  4  2]
 [ 5  8  3  8]
 [ 9  0 11  3]
 [ 2  0  7  7]]
Accuracy: 0.4946236559139785
[[10  5  4  6]
 [13 12  0 13]
 [ 1  1  2  1]
 [ 8  5  4  8]]
Accuracy: 0.34408602150537637
[[12  4  2 12]
 [ 5  3  0  5]
 [ 4  0  3  2]
 [14  7  3 16]]
Accuracy: 0.3695652173913043
[[16  6  0  9]
 [ 3  5  0  3]
 [ 4  0  7  1]
 [ 9  4  0 25]]
Accuracy: 0.5760869565217391
Average accuracy:  0.4493454885460496
k=4
[[23  3  4 10]
 [ 3 10  1  6]
 [ 4  8 22  2]
 [ 9  3  3  5]]
Accuracy: 0.5172413793103449
[[19  5  6  6]
 [ 5 14  8  9]
 [ 1  0 18  1]
 [ 7  3  7  7]]
Accuracy: 0.5
[[10  7  4 12]
 [ 8  8  1 14]
 [ 1  1  3  3]
 [16  9  2 17]]
Accuracy: 0.3275862068965517
[[15 10  0 15]
 [ 2  9  0  4]
 [ 6  0  5  3]
 [ 5  9  1 31]]
Accuracy: 0.5217391304347826
Average accuracy:  0.46664167916041976
